# Build simplest supercell

In [1]:
from ase.visualize import view
from ase.io import write

from VaspDefAnalysis.defect.builder import Supercell
from VaspDefAnalysis.utils.path_save_output import SaveOutput 

# Save or not output
SAVE = True

# Where it is saved the super cell 
save = SaveOutput('outputs/supercell/hybrid_hse06')

unit_cell_path = 'outputs/unitcell/structure_opt/POSCAR_hybrid_hse06'

# Directions
x_direction = 4
y_direction = 4 
z_direction = 2

# Save in 
dir_name=f'supercell_{x_direction}x{y_direction}x{z_direction}'

# Matrix trnslation
matrix_translation = [[x_direction, 0, 0], [0, y_direction, 0], [0, 0, z_direction]]

# Generte the supercell
supercell = Supercell.get_supercell(structure = unit_cell_path, transformation_matrix = matrix_translation )

if SAVE == True:
    # Export supercell into vasp
    name_save = 'supercell_'+ str(x_direction)+'x'+ str(y_direction)+'x'+ str(z_direction)+'.vasp'
    Data_id = save.data_path(data_name=name_save,dir_name=dir_name)
    
    # Write the infomation of the super cell in a text
    write(Data_id, supercell, vasp5=True, sort=True, direct=True)

# Number of atomos
supercell.get_global_number_of_atoms()

128

# Find the index cental atoms and the neribors to this 

In [7]:
from VaspDefAnalysis.utils.tool_pool import find_index_position_symbol_of_most_center_atom,find_indexs_positions_distances_symbols_inside_raduis

# The most centest atoms infomation
centest_atoms_inf =  find_index_position_symbol_of_most_center_atom(structure=supercell,species="N")

# Radius to consider surrounding neighbors of the defect (to select the first neighbor, it should be set slightly larger than the bonding length. For example, a radius of 2.0 Å for AlN, where the bonding length is approximately 1.87 Å).
radius = 2.0    # Å 

# Neighbors infoamtion
neighbors_inf = find_indexs_positions_distances_symbols_inside_raduis(structure=supercell,radius_centered_in_position=centest_atoms_inf["position"],radius=radius)

print(centest_atoms_inf)
print(neighbors_inf)

{'index': 78, 'position': array([4.65492123, 4.47920004, 6.86268444]), 'symbol': 'N'}
{'indexs': [45, 72, 77, 85], 'distances': [1.8853179080315023, 1.8979711905368788, 1.8853179085423546, 1.8853179080090048], 'posiitions': [array([3.10328082, 3.58336003, 7.44946067]), array([4.65492123, 4.47920004, 4.96471325]), array([6.20656163, 3.58336003, 7.44946067]), array([4.65492123, 6.27088005, 7.44946067])], 'symbols': ['Al', 'Al', 'Al', 'Al']}


# Make defect 

## Vacancy

In [5]:
from ase.visualize import view
from ase import io, Atoms
from ase.io import write, read

from VaspDefAnalysis.defect.builder import Makedefect
from defect.structure_comparator import StructureComparator

# Save or not output
SAVE =  True

defect = Makedefect(structure = supercell)
atom_inf = centest_atoms_inf 
vacancy_supercell = defect.make_vacancy(index = atom_inf["index"])

defect_analysis = StructureComparator(perfect_structure_file=supercell,defect_structure_file=vacancy_supercell,tolerance=1.0) 

defect_analysis.get_defect_information()

view(vacancy_supercell)


if SAVE == True:
    # Export supercell into vasp
    name_save = 'POSCAR-'+'V_'+atom_inf["symbol"]+'.vasp'
    save_data = save.data_path(data_name=name_save,dir_name=dir_name)
    
    # Write the infomation of the super cell in a text
    write(save_data, vacancy_supercell, vasp5=True, sort=True, direct=True)

Vacancy: Al, in the position: [0.375      0.63317302 0.49951891]


# Substitutional

In [8]:
from ase.visualize import view
from ase import io, Atoms
from ase.io import write, read

from VaspDefAnalysis.defect.builder import Makedefect
from defect.structure_comparator import StructureComparator

# Save or not output
SAVE = True

defect = Makedefect(structure = supercell)
substitutional_element = "C"
substitutional_defect = defect.make_substitutional(index = centest_atoms_inf["index"], new_element=substitutional_element)

defect_analysis = StructureComparator(perfect_structure_file=supercell,defect_structure_file=substitutional_defect,tolerance=1.0) 

defect_analysis.get_defect_information()

view(substitutional_defect)

if SAVE == True:
    # Export supercell into vasp
    
    name_save = 'POSCAR-'+substitutional_element+'_'+centest_atoms_inf["symbol"]+'.vasp'
    save_data = save.data_path(data_name=name_save,dir_name=dir_name)
    
    # Write the infomation of the super cell in a text
    write(save_data, substitutional_defect, vasp5=True, sort=True, direct=True)

Substitution: N replaced by C, in the position: [0.375      0.63317302 0.69048109]


# Antisite

In [9]:
from ase.visualize import view
from ase import io, Atoms
from ase.io import write, read

from VaspDefAnalysis.defect.builder import Makedefect
from defect.structure_comparator import StructureComparator

# Save or not output
SAVE = True


defect = Makedefect(structure = supercell)
antisite_defect = defect.make_antisite(index1 = centest_atoms_inf["index"],index2 = neighbors_inf["indexs"][0])

defect_analysis = StructureComparator(perfect_structure_file=supercell,defect_structure_file=antisite_defect) 

defect_analysis.get_defect_information()

view(antisite_defect)

if SAVE == True:
    # Export supercell into vasp
    
    name_save = 'POSCAR-'+centest_atoms_inf["symbol"]+'_'+neighbors_inf["symbols"][0]+'-'+neighbors_inf["symbols"][0]+'_'+centest_atoms_inf["symbol"]+'.vasp'
    save_data = save.data_path(data_name=name_save,dir_name=dir_name)
    
    # Write the infomation of the super cell in a text
    write(save_data, antisite_defect, vasp5=True, sort=True, direct=True)

Complex:
Substitutions:
Al replaced by N at [0.25       0.4776709  0.74951891]
N replaced by Al at [0.375      0.63317302 0.69048109]


# Make dimero

In [11]:
from ase.visualize import view
from ase import io, Atoms
from ase.io import write, read

from VaspDefAnalysis.defect.builder import Makedefect
from defect.structure_comparator import StructureComparator

# Save or not output
SAVE = True


defect = Makedefect(structure = supercell)
substitutional_element = "C"
dimero_defect = defect.make_dimero(index1=centest_atoms_inf["index"],index2=neighbors_inf["indexs"][0],element=substitutional_element)

defect_analysis = StructureComparator(perfect_structure_file=supercell,
                                 defect_structure_file=dimero_defect) 

defect_analysis.get_defect_information()

view(dimero_defect)

if SAVE == True:
    # Export supercell into vasp
    
    name_save = 'POSCAR-'+substitutional_element+'_'+centest_atoms_inf["symbol"]+'-'+substitutional_element+'_'+neighbors_inf["symbols"][0]+'.vasp'
    save_data = save.data_path(data_name=name_save,dir_name=dir_name)
    
    # Write the infomation of the super cell in a text
    write(save_data, dimero_defect, vasp5=True, sort=True, direct=True)

Complex:
Substitutions:
Al replaced by C at [0.25       0.4776709  0.74951891]
N replaced by C at [0.375      0.63317302 0.69048109]


# Make divacancy

In [12]:
from ase.visualize import view
from ase import io, Atoms
from ase.io import write, read

from VaspDefAnalysis.defect.builder import Makedefect
from defect.structure_comparator import StructureComparator

# Save or not output
SAVE =  True

defect = Makedefect(structure = supercell)
atom_inf1 = centest_atoms_inf
atom_inf2 = neighbors_inf 
vacancy_supercell = defect.make_divacancy(index1 = atom_inf1["index"],index2=atom_inf2["indexs"][0])

defect_analysis = StructureComparator(perfect_structure_file=supercell,defect_structure_file=vacancy_supercell,tolerance=1.0) 

defect_analysis.get_defect_information()

view(vacancy_supercell)


if SAVE == True:
    # Export supercell into vasp
    name_save = 'POSCAR-'+'V_'+atom_inf1["symbol"]+"-"+'V_'+atom_inf2["symbols"][0]+'.vasp'
    save_data = save.data_path(data_name=name_save,dir_name=dir_name)
    
    # Write the infomation of the super cell in a text
    write(save_data, vacancy_supercell, vasp5=True, sort=True, direct=True)

128
Complex:
Vacancies:
Al at [0.25       0.4776709  0.74951891]
N at [0.375      0.63317302 0.69048109]


In [15]:

from ase import Atoms
import numpy as np

# Example: Create a structure with some atoms
# Assume atoms is an ASE Atoms object with a defined cell and positions
atoms = Atoms('H2O', positions=[[0, 0, 0], [1.345459585, 0, 1.45563], [1, 1, 4]], cell=[[10, 0, 0], [5 ,10, 0], [0, 0, 10]])

# Get the Cartesian coordinates of an atom (e.g., the first atom)
cartesian_coordinates = atoms.positions[1]

print(atoms.cell)
print(np.linalg.inv(atoms.cell))
# Convert to fractional coordinates
fractional_coordinates = np.dot(np.linalg.inv(atoms.cell), cartesian_coordinates)

cartesian_coordinates_new = np.dot(atoms.cell, fractional_coordinates)

print(f"Cartesian Coordinates: {np.linalg.norm(cartesian_coordinates)}")
print(f"Fractional Coordinates: {np.linalg.norm( fractional_coordinates)}")
print("------------------------------")
print(f"Cartesian Coordinates: {np.linalg.norm(cartesian_coordinates_new)}")

Cell([[10.0, 0.0, 0.0], [5.0, 10.0, 0.0], [0.0, 0.0, 10.0]])
[[ 0.1   0.    0.  ]
 [-0.05  0.1   0.  ]
 [ 0.    0.    0.1 ]]
Cartesian Coordinates: 1.9822008454665667
Fractional Coordinates: 0.20932476120816346
------------------------------
Cartesian Coordinates: 1.9822008454665667
